(1) Import 使用的模組進來 , Import library

In [1]:
import pandas as pd
import numpy as np
import csv
import os
import sys
import pickle

In [2]:
#from NameModules.fortune_calculator import moe_data_dict, special_word_dict
from NameModules.fortune_calculator import stroke_total
from NameModules.fortune_calculator import stroke_outside
from NameModules.fortune_calculator import stroke_man
from NameModules.fortune_calculator import stroke_heaven
from NameModules.fortune_calculator import stroke_earth
from NameModules.fortune_calculator import test_name_Fortune_telling
from NameModules.fortune_calculator import get_talent_type
from NameModules.fortune_calculator import get_talent_state
from NameModules.fortune_calculator import get_stroke_state
from NameModules.fortune_calculator import Get_stroke
from NameModules.share_lib import is_chinese, is_number, PrintException
from NameModules.Taiwan_name_seperate import checkLastName, GetLastName, GetFirstName, get_LastName_from_FN, is_biFirstName


'''
checkLastName: 檢查名字的姓氏有無在百家姓的list中
GetLastName: 取姓氏
GetLastName: 取名字
get_LastName_from_FN:
is_biFirstName:是否是疊字名字 
'''
from NameModules.Taiwan_name_seperate import Taiwan_LastName_List, Taiwan_LastName_len1_List, Taiwan_LastName_len2_List
from NameModules.NameModule import contain_Simplified_character , contain_unreadable_character

'''
contain_Simplified_character: find名字是否含非繁體字會用之字的簡體字
contain_unreadable_character: 名字否含不可解讀的編碼的特殊字元
'''

from NameModules.share_lib import is_number,is_chinese, reduce_mem_usage
'''
reduce_mem_usage: 自動將df各欄的型態轉型成可接受的較小size 型態
'''

from gensim.models import word2vec

from hanziconv import HanziConv 


def NameClean(Name_df):
    ID_col = 'userID'
    if ID_col in Name_df.columns:
        #Drop duplicate name in FB source
        FB_Name_df = Name_df [ Name_df[ID_col].apply(lambda x: type(x)!=float) ]
        ori_len = len(FB_Name_df)
        FB_Name_df = FB_Name_df.drop_duplicates(subset= ID_col ,keep='first',inplace=False)
        print ('Drop duplicate FB name: from ',ori_len,'->',len(FB_Name_df),' drop:',ori_len-len(FB_Name_df))
        
        Name_df = Name_df[ Name_df[ID_col].apply(lambda x: type(x)==float) ]
        Name_df = pd.concat([Name_df, FB_Name_df], ignore_index=True)
    
    #Drop Message is not number
    ori_len = len(Name_df)
    Name_df  = Name_df[ Name_df.message.apply(lambda x:type(x)!=float) ]
    Name_df  = Name_df[ Name_df.message.apply(lambda x: is_number(x)) ]
    print ('Drop Message is not number: ',ori_len,'->',len(Name_df),' drop:',ori_len-len(Name_df))
    ori_len = len(Name_df)
    
    #Drop English name
    ori_len = len(Name_df)
    Name_df = Name_df[ Name_df.name.apply(lambda x: (is_chinese(x))) ]
    print ('Drop English name: from ',ori_len,'->',len(Name_df),' drop:',ori_len - len(Name_df))

    ori_len = len(Name_df)
    #Drop last name is not in Taiwan all last name
    Name_df = Name_df[Name_df.name.apply(lambda x: ( checkLastName(x)))]
    print ('Dron last name is not in Taiwan last name list :',ori_len,'->',len(Name_df),' drop:',ori_len - len(Name_df))
    ori_len = len(Name_df)

    Name_df['LastName'] = Name_df.name.apply(lambda x: (GetLastName(x)))
    Name_df['FirstName'] = Name_df.name.apply(lambda x: (GetFirstName(x)))

    #Drop First name is longer than 3 
    Name_df = Name_df[Name_df.FirstName.apply(lambda x: len(x)<3 )]
    print ('Drop First name is longer than 3  :',ori_len,'->',len(Name_df),' drop:',ori_len - len(Name_df))
    ori_len = len(Name_df)
    
    return Name_df


def Read_Saved_Files():
    pkl_data_path = './pkl_data/'
    '''
    w2v phonetic
    '''
    # 2XX萬名字的涵蓋的中文字 List,共8578種字, 如果把character 變成 編號的時候可以用
    Totalname_list = ''
    with open( pkl_data_path + 'Totalname_list.pkl', 'rb') as handle:
        Totalname_list = pickle.loads(handle.read())

    # 萌典的本體 dict
    moe_data_dict = {}
    with open( pkl_data_path +'moe_data_dict.pkl', 'rb') as handle:
        moe_data_dict = pickle.loads(handle.read())

    # 存同義字或是詞的dict
    common_dict = {}
    with open( pkl_data_path + 'common_dict.pkl', 'rb') as handle:
        common_dict = pickle.loads(handle.read())

    '''
    phonetic
    '''
    son_in_list = []
    with open( pkl_data_path + 'son_in_list.pkl', 'rb') as handle:
        son_in_list = pickle.loads(handle.read())

    mu_in_list = []
    with open( pkl_data_path + 'mu_in_list.pkl', 'rb') as handle:
        mu_in_list = pickle.loads(handle.read())

    special_word_dict = {}
    with open( pkl_data_path + 'special_word_dict.pkl', 'rb') as handle:
        special_word_dict = pickle.loads(handle.read())

    return Totalname_list, moe_data_dict, common_dict, son_in_list, mu_in_list, special_word_dict

def get_x_feature(feature_list , columns):
    f = []
    if 'gender' in feature_list:
        f+=['Male_prob' , 'Female_prob']
    
    if 'uni-gram' in feature_list:
        f+=['FN1','FN2']
        
    for col in columns:
        if 'w2v' in feature_list and '_wv_' in col:
            f+= [col]
        elif 'phonetic' in feature_list and ( 'muin_' in col or 'sonin_' in col):
            f+=[col]
        elif 'fortune_map' in feature_list and ("格_" in col or '三才_' in col):
            f+=[col]
        elif 'Zodiac' in feature_list and 'Zodiac_' in col:
            f+=[col]
        elif 'radical' in feature_list and 'radical_' in col:
            f+=[col]
            
    return f 

def add_word_vector(vector_model,word,n):
    if word in vector_model.wv:
        return vector_model.wv[word][n]
    else:
        if word in common_dict:
            if common_dict[word] in vector_model.wv:
                return vector_model.wv[common_dict[word]][n]
            else:
                return 0
        else:
            return 0
        
def Get_FN_WV( vector_model, FirstName, FN_loc ,WV_loc):
    FN =' '
    if len(FirstName)==2:
        if FN_loc == 1:FN=FirstName[0]
        else:FN=FirstName[1]
    elif FN_loc==2:
        FN=FirstName
    return add_word_vector( vector_model,FN,WV_loc)       

def character_to_index(name,n):
    if n==1:
        if name[0] in Totalname_list:
            return Totalname_list.index(name[0])
        else:
            Totalname_list.append(name[0])
            return Totalname_list.index(name[0])
    if n==2 and len(name)==2:
        if name[1] in Totalname_list:
            return Totalname_list.index(name[1])
        else:
            Totalname_list.append(name[1])
            return Totalname_list.index(name[1])
    return -1

def add_pin_in_column(character, mode):
    # mode 1 = sonin = consonant
    # mode 2 = muin = vowel

    if character == -1:
        return None
    
    #不在字典內的補齊
    specail_word_pinyin_dic = {'艶': 'yàn', '鳯': 'fèng', '恵': 'huì', '姈': 'líng', '寳': 'bǎo', '姫': 'jī', '鑅': 'róng',
                               "玂": "qí", "浤": "hóng", '煊': 'xuān', '斔': 'zhōng', '琜': 'lái', '苰': 'hóng', '玹': 'xuán', '姵': 'pèi', '妏': 'wèn',
                               '妘': 'yún', '珺': 'jùn', '媗': 'xuān', '彣': 'wén', '玹': 'xuán', '瀞': 'jìng', '妡': 'xīn', '琁': 'xuán', '浤': 'hóng', '緁': 'jī',
                               '媜': 'zhēng', '姸': 'yán', '嬅': 'huà', '眞': 'zhēn', '廼': 'nǎi', '寛': 'kuān', '秝': 'lì', '蕥': 'yǎ', '汯': 'hóng', '逹': 'dá', '萓': 'yí',
                               '媃': 'róu', '孋': 'lí', '媁': 'wěi', '祤': 'yǔ', '媄': 'měi', '夆': 'fēng', '蒝': 'yuán', '嬣': 'níng', '砡': 'yù', '芠': 'wén',
                               '姳': 'mǐng', '蔆': 'líng', '菈': 'lā', '鍹': 'xuān', '榳': 'tíng', '錤': 'jī', '憓': 'huì', '潓': 'huì', '瓈': 'lí', '芛': 'wěi',
                               '峮': 'qún', '鋕': 'zhì', '姷': 'yòu', '兪': 'yú', '瑠': 'liú', '嫙': 'xuán', '珅': 'shēn', '暟': 'kǎi', '斈': 'xué', '煐': 'yīng', '淓': 'fāng', '瑨': 'jìn', '嬨': 'cí', '琹': 'qín', '珆': 'yí', '琣': 'pěi',
                               '娪': 'wú', '荺': 'yǔn', '爕': 'xiè', '玶': 'píng', '鋆': 'yún', '愼': 'shèn', '斳': 'qín', '瑈': 'róu', '澪': 'líng', '珦': 'xiàng', '妶': 'xián', '姃': 'zhēng', '薾': 'ěr', '溎': 'guì', '琄': 'xuàn', '琡': 'shū', '瑭': 'táng', '嫆': 'róng'
                               }
    #term = character
    term = Totalname_list[character]
    unkown_dict = {}
    try:
        if term not in moe_data_dict:
            term = HanziConv.toTraditional(term)

        if term == ' ':
            return None

        if term in moe_data_dict:
            for hete in moe_data_dict[term]['heteronyms']:
                if 'pinyin' in hete and moe_data_dict[term]['title'] != '啐':
                    word_p = hete['pinyin']

                    if '（' in (hete['pinyin']):
                        if '（讀音）' in hete['pinyin']:
                            word_p = hete['pinyin'].replace('（讀音）', '')
                        if '（語音）' in hete['pinyin']:
                            word_p = hete['pinyin'].replace('（語音）', '')
                        if '(' in word_p:
                            print(word_p+"!!")

                    for mu in mu_in_list:
                        if mu in word_p:
                            if mode == 'sonin':
                                return word_p[: word_p.index(mu)]
                            else:
                                return mu
            # 找不到字音，看是否是哪個字的異體字
            for hete in moe_data_dict[term]['heteronyms']:
                for define in hete['definitions']:
                    if '異體字' in define['def']:
                        d = define['def']
                        alt_term = d[d.index('「')+1: d.index('」')]
                        # print(alt_term,term)

                        for hete2 in moe_data_dict[alt_term]['heteronyms']:
                            if 'pinyin' in hete2 and moe_data_dict[alt_term]['title'] != '啐':

                                word_p = hete2['pinyin']
                                if '（' in (hete2['pinyin']):
                                    if '（讀音）' in hete2['pinyin']:
                                        word_p = hete2['pinyin'].replace(
                                            '（讀音）', '')
                                    if '（語音）' in hete2['pinyin']:
                                        word_p = hete2['pinyin'].replace(
                                            '（語音）', '')
                                    if '(' in word_p:
                                        print(word_p+"!!")

                                for mu in mu_in_list:
                                    if mu in word_p:
                                        if mode == 'sonin':
                                            return word_p[: word_p.index(mu)]
                                        else:
                                            return mu
            # print('在字典內但沒有拼音：',term)
        else:
            # print('不在字典內：',term)

            if term in special_word_dict:
                word_p = special_word_dict[term]['pinyin']
                for mu in mu_in_list:
                    if mu in word_p:
                        if mode == 'sonin':
                            return word_p[: word_p.index(mu)]
                        else:
                            return mu
            else:
                print('拼音不明：',term)
                if term not in unkown_dict:
                    unkown_dict[term] = 1
                else:
                    unkown_dict[term] += 1

#             if len( moe_df[moe_df.字詞名.apply(lambda x: x==term)])>0:
#                 print('不在字典內：',term)
        return unkown_dict
    except Exception as e:
        print(e)
        PrintException()

def add_W2V_feature( Name_df,w2v_Vector_number):
    ###############################################################
    #Add w2v
    #用17G的維基百科+幾十M的萌典範文訓練的w2v, 每個entry是一個字
    moe_model = word2vec.Word2Vec.load("./w2v_data/wiki_moe_100_model.bin")
    
    for i in range(0,w2v_Vector_number):
        Name_df['FN1_wv_'+str(i)]  = Name_df["FirstName"].apply(lambda x:  Get_FN_WV( moe_model , x, 1 , i))
        Name_df['FN2_wv_'+str(i)]  = Name_df["FirstName"].apply(lambda x:  Get_FN_WV( moe_model , x, 2 , i))
        
    Name_df , NA_list = reduce_mem_usage( Name_df )
    
    w2v_feature = get_x_feature( ['w2v'], Name_df.columns)
    print("w2v_feature len",len(w2v_feature))
    ################################################################ 
    return Name_df , w2v_feature

def add_phonetic_feature(Name_df):
    Name_df['FN1'] = Name_df.FirstName.apply(lambda x: character_to_index(x,1))
    Name_df['FN2'] = Name_df.FirstName.apply(lambda x: character_to_index(x,2))
    
    Name_df['FN1_sonin'] = Name_df.FN1.apply(lambda x:  add_pin_in_column(x,'sonin') )
    Name_df['FN1_muin'] = Name_df.FN1.apply(lambda x:  add_pin_in_column(x,'muin') )

    Name_df['FN2_sonin'] = Name_df.FN2.apply(lambda x:  add_pin_in_column(x,'sonin') )
    Name_df['FN2_muin'] = Name_df.FN2.apply(lambda x:  add_pin_in_column(x,'muin') )

    Name_df = pd.get_dummies(Name_df, columns=["FN1_sonin"])
    Name_df = pd.get_dummies(Name_df, columns=["FN1_muin"])
    Name_df = pd.get_dummies(Name_df, columns=["FN2_sonin"])
    Name_df = pd.get_dummies(Name_df, columns=["FN2_muin"])
    
    phonetic_feature = get_x_feature( ['phonetic'] , Name_df.columns)
    print('phonetic_feature len:',len(phonetic_feature))
    
    return Name_df , phonetic_feature

def add_fortune_map_feature(sampled_df):
    sampled_df['天格'] = sampled_df['LastName'].apply(lambda x: get_stroke_state(stroke_heaven (x)) )
    sampled_df['地格'] = sampled_df['LastName'].apply(lambda x: get_stroke_state(stroke_earth (x)) )
    sampled_df['人格'] = sampled_df.apply(lambda x: get_stroke_state(stroke_man (x)) ,axis = 1)
    sampled_df['外格'] = sampled_df.apply(lambda x: get_stroke_state(stroke_outside (x)) ,axis = 1)
    sampled_df['總格'] = sampled_df.apply(lambda x: get_stroke_state(stroke_total (x)) ,axis = 1)
    sampled_df['三才'] = sampled_df.apply(lambda x: get_talent_state(get_talent_type(stroke_heaven(
        x['LastName'])) + get_talent_type(stroke_earth(x['LastName'])) + get_talent_type(stroke_man(x))), axis=1)
    
    sampled_df = pd.get_dummies(sampled_df, columns=["天格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["地格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["人格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["外格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["總格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["三才"])

    fortune_map_feature_list = get_x_feature( ['fortune_map'] , sampled_df.columns)
    print("len on fortune_map_feature_list:",len(fortune_map_feature_list))
    return sampled_df , fortune_map_feature_list

def add_radical_column_age(character):
    if character in moe_data_dict:
        return moe_data_dict[character]['radical'] 
    elif character in special_word_dict :
        return special_word_dict[character]['radical']  
    else:
        return '不明'
        #return -1

def add_radical_feature(sampled_df):
    #FN1存成index
    sampled_df['FN1_radical'] = sampled_df['FN1'].apply(lambda x: add_radical_column_age( Totalname_list[x]))
    sampled_df['FN2_radical'] = sampled_df['FN2'].apply(lambda x: add_radical_column_age( Totalname_list[x]))
    sampled_df = pd.get_dummies(sampled_df, columns=["FN1_radical"])
    sampled_df = pd.get_dummies(sampled_df, columns=["FN2_radical"])

    Radical_feature_list = get_x_feature( ['radical'] , sampled_df.columns)
    print("len of Radical_feature_list: ",len(Radical_feature_list))
    return sampled_df , Radical_feature_list
    
def get_zodiac_from_birthyear(birthyear):
    zodiac_list = ['鼠', '牛', '虎', '兔', '龍', '蛇', '馬', '羊', '猴', '雞', '狗', '豬']
    return zodiac_list[divmod((birthyear - 4), 12)[1]]


def add_zodiac_feature(Name_df):
    Name_df['Zodiac'] = Name_df['message'].apply(lambda x: get_zodiac_from_birthyear(x))
    Name_df = pd.get_dummies(Name_df, columns=["Zodiac"])
    
    Zodiac_feature_list = get_x_feature("Zodiac" , Name_df )
    print("len of Zodiac_feature_list: ",len(Zodiac_feature_list))
    return Name_df , Zodiac_feature_list 

In [3]:
Totalname_list, moe_data_dict, common_dict, son_in_list, mu_in_list, special_word_dict = Read_Saved_Files()
def Name_df_Preprocess(Name_df ):
    Name_df = Name_df[[ 'name', 'BirthYear', 'FirstName', 'LastName','message']]
    
    #將object type 復原回int
    Name_df['BirthYear'] = Name_df.BirthYear.apply(lambda x: int(x))
    #Name_df['gender'] = Name_df.gender.apply(lambda x: int(float(x)))
    Name_df['message'] =  Name_df.message.apply(lambda x: int(x))

    #自動轉型
    Name_df , NA_list = reduce_mem_usage( Name_df)
    
    print("Name_df len",len(Name_df))
    Name_df = NameClean(Name_df)
    

    '''
    1 #用17G的維基百科+幾十M的萌典範文訓練的w2v, 每個entry是一個字
    '''
    
    Name_df , w2v_feature = add_W2V_feature( Name_df,w2v_Vector_number = 100)
    
    ################################################################
    '''
    2. 加入拼音feature  - one-hot encoding
    拼音是根據萌典與新華字典切出來的
    子音 = consonant = sonin 
    母音 = vowel = muin
    之前是隨意取名
    '''
    Name_df , phonetic_feature = add_phonetic_feature( Name_df)
    
    '''
    3. Fortune map - one-hot encoding
    '''
    Name_df , fortune_map_feature_list = add_fortune_map_feature ( Name_df )
    
    '''
    4. Character - one-hot encoding
    radical
    '''   
    Name_df , radical_feature_list = add_radical_feature ( Name_df )

    '''
    5. Zodiac_feature - one-hot encoding
    '''
    Name_df, Zodiac_feature_list = add_zodiac_feature( Name_df )
    
    return Name_df

In [4]:
FileName = "W_gnder_RFC_model.pkl"
with open('./TrainedModel/'+ FileName , 'rb') as handle:
    gender_model =  pickle.loads(handle.read())
FileName = "W_feature_list.pkl"
with open('./TrainedModel/'+ FileName , 'rb') as handle:
    gender_x_feature =  pickle.loads(handle.read())
    
def  add_gender_feature(Name_df):
    predict_proba = gender_model.predict_proba( Name_df[gender_x_feature])
    Male_prob_list = []
    Female_prob_list = []
    for prob in predict_proba:
        Female_prob_list+=[ prob[0] ]
        Male_prob_list+=[ prob[1] ]
    Name_df['Male_prob'] = Male_prob_list
    Name_df['Female_prob'] = Female_prob_list
    return Name_df

(2)讀檔 Read Files

(2-1A) Read raw Taiwanese dataset

In [4]:
Path = './NameData/Real_Name_data.csv'
Name_df = pd.read_csv(Path, dtype='str')
#Name_df [['name', 'BirthYear','FirstName','LastName','gender','message','userID','WritingNumber']].to_csv("./NameData/Total_Name.csv")

(2-1B) Read sampled dataset as base for comparing methods 

In [5]:
with open('./NameData/thesis_experiment/TaiwanNames_real_Name/final_real_name_df', 'rb') as handle:
     small_df = pickle.loads(handle.read())

In [6]:
with open('./NameData/thesis_experiment/TaiwanNames_real_Name/test_index', 'rb') as handle:
     test_index = pickle.loads(handle.read())
with open('./NameData/thesis_experiment/TaiwanNames_real_Name/train_index', 'rb') as handle:
     train_index = pickle.loads(handle.read())

In [7]:
small_df

,FirstName,message,name,FN1,FN2,FN1_muin,FN2_muin,BirthYear,GuessedGender,BFN12P,...,FirstName1_ratio_6,FirstName2_ratio_6,FirstName1_ratio_7,FirstName2_ratio_7,FirstName1_ratio_8,FirstName2_ratio_8,FirstName1_ratio_9,FirstName2_ratio_9,FirstName1_ratio_10,FirstName2_ratio_10
0,秀珠,1947,吳秀珠,70,596,iù,ū,1,0,1,...,0.014783,0.004958,0.005215,0.001011,0.004419,0.000620,0.002103,0.000166,0.001614,0.000071
1,豐博,1945,林豐博,277,72,ēng,ó,1,1,6,...,0.002408,0.001052,0.000821,0.000999,0.000986,0.002212,0.000701,0.002531,0.000680,0.003256
2,義勝,1945,康義勝,650,156,ì,èng,1,1,1,...,0.003228,0.003177,0.001718,0.001045,0.001546,0.002305,0.000899,0.001726,0.000665,0.001187
3,麗,1946,吳麗,-1,203,-1,ì,1,0,6,...,0.002688,0.015062,0.001453,0.002185,0.004345,0.002212,0.007338,0.001254,0.012524,0.000856
5,世慶,1947,林世慶,537,269,ì,ìng,1,1,6,...,0.004557,0.004066,0.004241,0.001383,0.003323,0.002356,0.002243,0.001752,0.001613,0.001902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320775,月鳳,1974,陳月鳳,706,598,uè,èng,6,0,6,...,0.003987,0.005077,0.001087,0.001245,0.000784,0.001598,0.000459,0.000935,0.000253,0.000493
320776,麗君,1974,吳麗君,203,153,ì,ūn,6,0,6,...,0.015062,0.006155,0.002185,0.007981,0.002212,0.013105,0.001254,0.009835,0.000856,0.007666
320777,玉淑,1974,陳玉淑,211,348,ù,ú,6,0,6,...,0.014855,0.022527,0.006995,0.007755,0.005954,0.005989,0.003115,0.003238,0.002560,0.001496
320778,明珠,1974,陳明珠,159,596,íng,ū,6,0,1,...,0.016520,0.004958,0.012100,0.001011,0.009044,0.000620,0.005848,0.000166,0.004553,0.000071


In [87]:
test_index

Int64Index([139458, 160569, 273443, 210241, 276709, 236523, 118678, 173062,
            189727, 265777,
            ...
            270402, 126974,   5332, 117560,  65924, 148607,  58561, 259018,
             89498, 194784],
           dtype='int64', length=90730)

In [8]:
small_df, Zodiac_feature_list = add_zodiac_feature( small_df )

len of Zodiac_feature_list:  12


(3) Construct features for training/test 

In [5]:
Name_df = Name_df[[ 'name', 'BirthYear', 'FirstName', 'LastName', 'gender',
       'message']]

In [8]:
#將object type 復原回int
Name_df['BirthYear'] = Name_df.BirthYear.apply(lambda x: int(x))
Name_df['gender'] = Name_df.gender.apply(lambda x: int(float(x)))
Name_df['message'] =  Name_df.message.apply(lambda x: int(x))

'''
type_dic = {}
for w2v_col in get_x_feature(['w2v'],Name_df):
    type_dic[ w2v_col ] = 'float32'

for col in get_x_feature(['phonetic','fortune_map','Zodiac','radical'],Name_df):
    type_dic[ col ] = 'int8'
'''

NameError: name 'Name_df' is not defined

(3-1) Resample train/test data,先Sample出每年齡層約3.2萬筆的名字再來加入其他feature,
反正都是feature都是用名字gen出來的,
直接存一個215萬x 2.300 col的df來做n-th folder太吃記憶體

In [13]:
from collections import Counter
Counter(Name_df.BirthYear.tolist()).most_common()

[(9, 444319),
 (8, 385892),
 (10, 284558),
 (7, 262114),
 (3, 60305),
 (4, 56396),
 (2, 54930),
 (5, 32241),
 (1, 31435),
 (6, 24284),
 (0, 15711),
 (11, 6621),
 (12, 2037),
 (13, 5)]

In [14]:
Name_df.head()

,name,BirthYear,FirstName,LastName,gender,message
0,丁承先,0,承先,丁,1,1940
1,丁昞原,0,昞原,丁,1,1940
2,方九龍,0,九龍,方,1,1940
3,方大錚,0,大錚,方,1,1940
4,方文隆,0,文隆,方,1,1940


In [25]:
'''
本來是用這段用到的BalanceCascade來sample,不過這函式不能用了,改用下面的方式來sample看看
from imblearn.ensemble import BalanceCascade
# Apply Balance Cascade method
bc = BalanceCascade( return_indices = True , classifier = 'random-forest')
X_resampled, y_resampled,indice = bc.fit_sample(train_compare_df[x_prob_feature],train_compare_df['compare_result'])
'''
def Sample_Name_df(df, sample_Number, BirthYear_list , ignore_index):
    sampled_df = pd.DataFrame()
    for BirthYear in BirthYear_list:
        sampled_df = pd.concat([sampled_df, df[df["BirthYear"].apply(lambda x: x == BirthYear)].sample(
            n=sample_Number, frac=None, replace=True, weights=None, random_state=np.random.RandomState(), axis=0)], ignore_index= ignore_index)
    return sampled_df

In [7]:
#1945~ 1994
BirthYear_list = [  i for i in range(1,11)]
BirthYear_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [26]:
#先用原論文數字30243,我當初篩過後,最少的 (1, 31419)是 30243
sample_Number = 30243
sampled_df = Sample_Name_df(Name_df, sample_Number, BirthYear_list, True)

In [37]:
sampled_df

,name,BirthYear,FirstName,LastName,gender,message
0,楊慈雲,1,慈雲,楊,7,1945
1,羅都容,1,都容,羅,1,1947
2,楊美英,1,美英,楊,0,1947
3,林崇安,1,崇安,林,1,1946
4,陳仲桐,1,仲桐,陳,7,1948
...,...,...,...,...,...,...
302425,廖有棚,10,有棚,廖,7,1992
302426,莊勝富,10,勝富,莊,7,1992
302427,徐上揚,10,上揚,徐,7,1992
302428,蘇玟瑄,10,玟瑄,蘇,7,1992


(4)Preparing for traing model 

(4-1) import fucntion

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [8]:
## from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


def dataset_statistics(dataset):
    """
    Basic statistics of the dataset
    :param dataset: Pandas dataframe
    :return: None, print the basic statistics of the dataset
    """
    print(dataset.describe())

# http://dataaspirant.com/2017/06/26/random-forest-classifier-python-scikit-learn/
# Split train and test


def split_dataset(dataset, train_percentage, feature_headers, target_header):
    """
    Split the dataset with train_percentage
    :param dataset:
    :param train_percentage:
    :param feature_headers:
    :param target_header:
    :return: train_x, test_x, train_y, test_y
    """

    # Split dataset into train and test dataset
    train_x, test_x, train_y, test_y = train_test_split(dataset[feature_headers], dataset[target_header],
                                                        train_size=train_percentage)
    return train_x, test_x, train_y, test_y


def random_forest_classifier(features, target, estimators_num=32,  min_samples_leaf_num=1):
    """
    To train the random forest classifier with features and target data
    :param features:
    :param target:
    :return: trained random forest classifier
    """
    clf = RandomForestClassifier(
        n_estimators= estimators_num, n_jobs=4, min_samples_leaf= min_samples_leaf_num , random_state=42)
    print('estimators_num = ' , estimators_num , 'min_samples_leaf_num = ' , min_samples_leaf_num ,"Training Data len = ", len(features))
    clf.fit(features, target)
    return clf


def random_forest_Regressor(features, target):
    """
    To train the random forest classifier with features and target data
    :param features:
    :param target:
    :return: trained random forest classifier
    """
    clf = RandomForestRegressor(n_estimators=32, n_jobs=4, min_samples_leaf=1)
    clf.fit(features, target)
    return clf


In [9]:
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn import metrics

(4-2) Make Feature Combinations

In [14]:
feature_list = ['w2v', 'phonetic','fortune_map','Zodiac','radical','gender','uni-gram']

In [13]:
feature_list_gender = ['w2v', 'phonetic','fortune_map','Zodiac','radical','uni-gram']

In [15]:
from itertools import chain, combinations

def get_all_combinations(feature_list):
    feature_combinations = [ x for x in feature_list]
    for i in range(2 , len(feature_list)+1):
        for combo in ( combinations(feature_list , r= i) ):
            temp =[]
            for f in combo:
                temp+=[f]
            feature_combinations += [temp]
    return feature_combinations

In [16]:
feature_combinations = get_all_combinations(feature_list)

In [17]:
x_feature = get_x_feature (['uni-gram'] , Name_df.columns)

NameError: name 'Name_df' is not defined

(4-3) 嘗試調整 - W2V 做normalize 

In [15]:
def _normalize(X, train = True, specified_column = None, X_mean = None, X_std = None):
    # This function normalizes specific columns of X.
    # The mean and standard variance of training data will be reused when processing testing data.
    #
    # Arguments:
    #     X: data to be processed
    #     train: 'True' when processing training data, 'False' for testing data
    #     specific_column: indexes of the columns that will be normalized. If 'None', all columns
    #         will be normalized.
    #     X_mean: mean value of training data, used when train = 'False'
    #     X_std: standard deviation of training data, used when train = 'False'
    # Outputs:
    #     X: normalized data
    #     X_mean: computed mean value of training data
    #     X_std: computed standard deviation of training data

    if specified_column == None:
        #np.arange(X.shape[1])只拿的到pd的shape  [0~n],不能拿到pd的欄位名稱
        specified_column = np.arange(X.shape[1])
        #specified_column = X.columns
    if train:
        #X_mean為一個list放每行的mean
        X_mean = np.mean(X[:, specified_column] ,0).reshape(1, -1)
        X_std  = np.std(X[:, specified_column], 0).reshape(1, -1)

    X[:,specified_column] = (X[:, specified_column] - X_mean) / (X_std + 1e-8)
     
    return X, X_mean, X_std

In [41]:
def w2v_normalize(sampled_df):
    x_feature = get_x_feature ( ['w2v'] , Name_df.columns)
    w2v_np = sampled_df[ x_feature ].to_numpy()
    # Normalize training and testing data
    w2v_np, X_mean, X_std = _normalize(w2v_np, train = True)
    
    idx = 0
    for i in range(100):
        for j in range(1,3):
            col = ("FN{}_wv_{}".format(j,i))
            #print(col)
            FN_w2vX =[] 
            for vector_list in (w2v_np):
                FN_w2vX+=[ vector_list[idx]  ]
            idx+=1
            sampled_df[col] = FN_w2vX
    return sampled_df

In [232]:
#未normalized
sampled_df[ x_feature ].describe()

,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,FN1_wv_4,FN2_wv_4,...,FN1_wv_95,FN2_wv_95,FN1_wv_96,FN2_wv_96,FN1_wv_97,FN2_wv_97,FN1_wv_98,FN2_wv_98,FN1_wv_99,FN2_wv_99
count,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,...,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000
mean,-1.149561,-1.142548,0.335170,0.984800,-1.303121,-1.285600,0.969382,1.649082,0.932246,0.639874,...,0.852125,0.855012,-0.818223,-1.027274,0.639393,0.495382,0.975133,0.860273,-1.179080,-1.059375
std,3.492488,3.323843,3.474877,3.337498,3.092279,3.200081,3.298764,3.259781,3.352327,3.217403,...,3.348929,3.137315,3.328003,3.158895,3.503989,3.088225,2.894282,2.827303,3.368849,3.369553
min,-14.552404,-14.552404,-15.155359,-15.155359,-10.770514,-12.120364,-13.059869,-13.059869,-19.813513,-19.813513,...,-13.671687,-15.675972,-11.840435,-13.274806,-10.960847,-10.634512,-13.096807,-13.096807,-11.716203,-11.936399
25%,-3.428356,-3.267654,-1.943416,-1.286454,-3.024722,-3.490335,-0.700949,-0.314655,-0.800308,-1.321648,...,-0.926266,-0.984259,-3.091983,-3.060984,-1.220916,-1.220916,-0.894890,-1.018355,-3.538004,-3.236742
50%,-0.849926,-1.173061,0.181308,1.185356,-1.684384,-1.669534,1.110161,1.810379,0.969153,0.752990,...,0.866414,1.216405,-0.612317,-1.291795,0.270797,0.459639,0.671934,0.629546,-0.906662,-1.021606
75%,0.880691,0.837845,2.884024,3.441082,0.328431,0.566860,3.235934,3.917055,3.014434,2.652030,...,2.981780,2.909477,1.220726,0.991717,2.560374,2.222106,2.960142,2.960142,0.654513,0.882206
max,13.809453,13.809453,13.278703,12.627606,12.577226,13.612901,15.600971,15.600971,12.603272,12.603272,...,13.278911,13.278911,17.067118,17.067118,15.014699,14.567921,13.154280,13.154280,14.519915,14.381725


In [41]:
sampled_df[x_feature]

,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,FN1_wv_4,FN2_wv_4,...,FN1_wv_95,FN2_wv_95,FN1_wv_96,FN2_wv_96,FN1_wv_97,FN2_wv_97,FN1_wv_98,FN2_wv_98,FN1_wv_99,FN2_wv_99
0,-0.082778,-5.796417,4.782166,-0.217218,-3.571659,-5.335327,-8.585439,0.896868,6.623681,0.994031,...,0.497893,2.754097,-6.471545,0.587246,-5.607302,-2.159950,4.919923,0.551634,-4.087385,-2.308148
1,-1.442101,0.381964,1.215422,2.645488,-1.004735,-1.196166,0.030470,4.425450,0.634333,-0.119361,...,1.299436,-0.530058,0.151594,1.345441,-1.183616,0.623379,-0.605316,-0.215416,0.720008,5.855138
2,0.000000,-3.596860,0.000000,-5.960489,0.000000,-0.759137,0.000000,-0.892981,0.000000,-1.062888,...,0.000000,1.683001,0.000000,4.143760,0.000000,4.756191,0.000000,-3.617392,0.000000,-7.906404
3,3.789320,-2.980340,-1.893114,0.727061,8.631498,2.532944,-1.192231,8.816960,0.861374,1.124468,...,3.327790,0.101130,1.703527,6.744654,-3.576160,2.269994,4.723351,4.592242,2.390387,-3.591447
4,3.813373,-4.719679,-0.937114,2.741030,-2.302430,-1.445924,1.900387,-2.508925,4.940345,0.752990,...,-0.926266,2.260610,5.119319,-0.142316,3.783037,-2.275315,1.980577,-1.235366,-2.943175,0.554239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138496,3.107082,1.373621,0.181308,7.103828,-1.789464,-1.372236,1.171252,-0.022872,1.328304,-3.410328,...,0.086999,6.691445,-0.047737,2.491465,-1.211868,-0.915223,1.569784,-1.038645,-4.228339,3.407803
138497,-0.024527,-0.138811,1.429394,3.504970,-5.721419,-0.308425,-1.453798,1.116156,4.826768,0.447931,...,6.056461,-1.345802,4.155251,1.048379,6.082980,-0.555668,-3.041093,1.900405,1.069293,0.627254
138498,4.559606,6.068225,2.725899,-2.063284,-2.807246,-0.077769,4.147959,1.140175,1.654552,-4.686434,...,-1.934218,-7.699350,4.781729,-6.722366,3.168396,2.253842,3.530610,-1.759062,-1.181018,2.278829
138499,0.000000,-2.398280,0.000000,1.651978,0.000000,-1.263631,0.000000,3.889518,0.000000,-1.008286,...,0.000000,4.664589,0.000000,-2.361263,0.000000,5.832688,0.000000,5.729279,0.000000,-0.815454


In [ ]:
sampled_df = w2v_normalize(sampled_df)

In [53]:
sampled_df[x_feature]

,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,FN1_wv_4,FN2_wv_4,...,FN1_wv_95,FN2_wv_95,FN1_wv_96,FN2_wv_96,FN1_wv_97,FN2_wv_97,FN1_wv_98,FN2_wv_98,FN1_wv_99,FN2_wv_99
0,0.305452,-1.400152,1.279761,-0.360157,-0.733616,-1.265512,-2.896495,-0.230757,1.697763,0.110076,...,-0.105775,0.605324,-1.698719,0.511105,-1.782745,-0.859828,1.362965,-0.109164,-0.863296,-0.370606
1,-0.083763,0.458661,0.253320,0.497587,0.096494,0.027948,-0.284626,0.851707,-0.088868,-0.235978,...,0.133569,-0.441484,0.291412,0.751125,-0.520268,0.041447,-0.546061,-0.380466,0.563722,2.052064
2,0.329154,-0.738399,-0.096455,-2.080995,0.421413,0.164516,-0.293863,-0.779829,-0.278090,-0.529237,...,-0.254448,0.263917,0.245861,1.636982,-0.182476,1.379700,-0.336918,-1.583729,0.349996,-2.032036
3,1.414149,-0.552914,-0.641258,-0.077225,3.212729,1.193269,-0.655282,2.198891,-0.021141,0.150617,...,0.739243,-0.240296,0.757739,2.460340,-1.203077,0.574640,1.295047,1.319980,1.059555,-0.751459
4,1.421036,-1.076207,-0.366139,0.526214,-0.323164,-0.050100,0.282229,-1.275553,1.195621,0.035158,...,-0.531035,0.448027,1.784122,0.280149,0.897165,-0.897184,0.347391,-0.741218,-0.523651,0.478883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138496,1.218804,0.757008,-0.044278,1.833425,-0.157277,-0.027073,0.061196,-0.512906,0.118145,-1.258847,...,-0.228470,1.860334,0.231517,1.113919,-0.528331,-0.456770,0.205458,-0.671639,-0.905137,1.325753
138497,0.322131,0.301982,0.314897,0.755110,-1.428821,0.305360,-0.734575,-0.163486,1.161741,-0.059658,...,1.554035,-0.701499,1.494437,0.657084,1.553546,-0.340342,-1.387646,0.367890,0.667403,0.500552
138498,1.634705,2.169416,0.688006,-0.913288,-0.486415,0.377439,0.963569,-0.156118,0.215465,-1.655474,...,-0.832013,-2.726661,1.682683,-1.802881,0.721753,0.569410,0.882943,-0.926446,-0.000575,0.990700
138499,0.329154,-0.377797,-0.096455,0.199904,0.421413,0.006865,-0.293863,0.687299,-0.278090,-0.512266,...,-0.254448,1.214284,0.245861,-0.422298,-0.182476,1.728283,-0.336918,1.722144,0.349996,0.072390


In [58]:
pd.options.display.float_format = '{:,.4f}'.format

In [59]:
#已normalized
sampled_df[ x_feature ].describe()

,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,FN1_wv_4,FN2_wv_4,...,FN1_wv_95,FN2_wv_95,FN1_wv_96,FN2_wv_96,FN1_wv_97,FN2_wv_97,FN1_wv_98,FN2_wv_98,FN1_wv_99,FN2_wv_99
count,"138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000",...,"138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000"
mean,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000,0.0000,-0.0000,0.0000,...,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,-0.0000,0.0000,0.0000,-0.0000
std,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
min,-3.8376,-4.0345,-4.4579,-4.8360,-3.0616,-3.3858,-4.2529,-4.5123,-6.1885,-6.3571,...,-4.3369,-5.2692,-3.3120,-3.8772,-3.3106,-3.6040,-4.8620,-4.9366,-3.1278,-3.2280
25%,-0.6525,-0.6394,-0.6557,-0.6805,-0.5567,-0.6890,-0.5064,-0.6024,-0.5168,-0.6097,...,-0.5310,-0.5863,-0.6832,-0.6438,-0.5309,-0.5558,-0.6461,-0.6645,-0.7002,-0.6462
50%,0.0858,-0.0092,-0.0443,0.0601,-0.1233,-0.1200,0.0427,0.0495,0.0110,0.0352,...,0.0043,0.1152,0.0619,-0.0837,-0.1052,-0.0116,-0.1048,-0.0816,0.0809,0.0112
75%,0.5813,0.5958,0.7335,0.7360,0.5276,0.5789,0.6871,0.6957,0.6211,0.6254,...,0.6359,0.6549,0.6127,0.6391,0.5482,0.5591,0.6858,0.7427,0.5443,0.5762
max,4.2832,4.4984,3.7249,3.4885,4.4887,4.6557,4.4355,4.2800,3.4815,3.7184,...,3.7107,3.9601,5.3742,5.7281,4.1026,4.5569,4.2080,4.3483,4.6601,4.5826


(5) Experiment - Train feature combination for RFC

In [18]:
import random
def get_test_name_dict_list(test_index,data_df,year_column):
    test_name_dict_list = {}
    FN_list = data_df.loc[test_index]['FirstName'].tolist()
    #year- message / birthyear
    Year_list = data_df.loc[test_index][year_column].tolist()
    for index in range(len(FN_list)):
        FN = FN_list[index]
        year = Year_list[index]
        #print(year)
        if FN in test_name_dict_list:
            if year  not in test_name_dict_list[FN]:
                test_name_dict_list[FN][year] =1
            else:
                test_name_dict_list[FN][year]+=1
        else:
            test_name_dict_list[FN] = {}
            test_name_dict_list[FN][year] =1
            
    return test_name_dict_list

def get_min_distance(a,b_list):
    smallest_dist = 999
    a=1940+a*5
    
    min_error_answer=0
    for number in b_list:
        if int(number)>a:
            year=a+4
        else:
            year=a
        dist = abs(year - int(number))
        if(dist<smallest_dist):
            smallest_dist=dist
    #print(a,b_list,smallest_dist)
    return smallest_dist

In [19]:
def Multi_RFC_metrics(  data_x , data_y , trained_model ):
        predicted = trained_model.predict(data_x)
        accuracy = accuracy_score(data_y,predicted)
        print("Accuracy :: ", accuracy)
        
        macro_precision = precision_score(data_y, predicted ,average='macro')
        micro_precision = precision_score(data_y, predicted,average='micro')
        print("macro precision_score :: ", macro_precision)
        print("micro precision_score :: ", micro_precision)
        
        macro_recall = recall_score(data_y, predicted ,average='macro')
        micro_recall = recall_score(data_y, predicted,average='micro')        
        print("Train recall_score :: ", macro_recall)
        print("Test recall_score :: ", micro_recall)
        
        macro_f1 = f1_score(data_y, predicted,average='macro')
        micro_f1 = f1_score(data_y, predicted,average='micro')
        print("macro F1_score :: ", macro_f1)
        print("micro F1_score :: ", micro_f1)


        return round( accuracy, 4) , round( macro_precision,4) , round(micro_precision ,4) ,\
                round(macro_recall,4) , round(micro_precision,4) \
                ,  round(macro_f1,4) ,round(micro_f1,4)



(5-1A)重新sample跟建feature, Construct features for training/test 

In [20]:
y_feature = 'BirthYear'

In [40]:
#先用原論文數字30243,我當初篩過後,最少的 (1, 31419)是 30243
sample_Number = 30243
sampled_df = Sample_Name_df(Name_df, sample_Number, BirthYear_list, True)

In [41]:
sampled_df 

,name,BirthYear,FirstName,LastName,gender,message
0,吳亮宏,1,亮宏,吳,7,1945
1,廖盛裕,1,盛裕,廖,1,1948
2,周宜民,1,宜民,周,1,1946
3,許宗銘,1,宗銘,許,1,1947
4,徐碧嶺,1,碧嶺,徐,7,1949
...,...,...,...,...,...,...
302425,柯瑀芊,10,瑀芊,柯,7,1990
302426,童鈺成,10,鈺成,童,7,1992
302427,李羿霆,10,羿霆,李,7,1993
302428,林斐婷,10,斐婷,林,7,1994


In [51]:
Counter(sampled_df.BirthYear.tolist()).most_common()

[(4, 30239),
 (6, 30239),
 (3, 30235),
 (5, 30235),
 (1, 30226),
 (2, 30189),
 (8, 30142),
 (9, 30071),
 (7, 30069),
 (10, 30022)]

In [42]:
sampled_df = Name_df_Preprocess(sampled_df)
sampled_df

<ipython-input-40-67e6eb58931e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Name_df['BirthYear'] = Name_df.BirthYear.apply(lambda x: int(x))
<ipython-input-40-67e6eb58931e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Name_df['message'] =  Name_df.message.apply(lambda x: int(x))
c:\Users\n7912\Desktop\code\TaiwanName\NameModules\share_lib.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

Memory usage of properties dataframe is : 11.536911010742188  MB
Memory usage is:  7.787454605102539  MB
This is  67.50034387759015 % of the initial size
Name_df len 302430
Drop Message is not number:  302430 -> 302430  drop: 0
Drop English name: from  302430 -> 301643  drop: 787
Dron last name is not in Taiwan last name list : 301643 -> 301643  drop: 0
Drop First name is longer than 3  : 301643 -> 301643  drop: 0
Memory usage of properties dataframe is : 470.3391122817993  MB
Memory usage is:  240.20376682281494  MB
This is  51.07033639143731 % of the initial size
w2v_feature len 200
phonetic_feature len: 314
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
禤 get stroke count failed!
len 

,name,BirthYear,FirstName,LastName,message,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,...,Zodiac_狗,Zodiac_猴,Zodiac_羊,Zodiac_虎,Zodiac_蛇,Zodiac_豬,Zodiac_雞,Zodiac_馬,Zodiac_鼠,Zodiac_龍
0,陳希銘,1,希銘,陳,1949,4.027665,-6.750647,0.026953,5.348444,-3.808539,...,0,0,0,0,0,0,0,0,0,0
1,謝月霞,1,月霞,謝,1948,2.137218,1.348958,1.206953,3.016007,2.698078,...,0,0,0,0,0,0,0,0,1,0
2,陳瑞玉,1,瑞玉,陳,1947,-1.118575,-1.203078,4.611594,5.645829,-7.272056,...,0,0,0,0,0,1,0,0,0,0
3,龔盛鑫,1,盛鑫,龔,1949,-0.986341,-3.385104,1.379764,2.407456,-4.399106,...,0,0,0,0,0,0,0,0,0,0
4,廉珠珠,1,珠珠,廉,1947,0.669739,0.669739,5.386898,5.386898,0.429497,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302425,劉家瑋,10,家瑋,劉,1992,-0.024527,-1.986696,1.429394,6.473218,-5.721419,...,0,1,0,0,0,0,0,0,0,0
302426,郭亮寬,10,亮寬,郭,1993,-7.725710,-4.826299,1.556864,-0.433100,-2.307863,...,0,0,0,0,0,0,1,0,0,0
302427,鄭婷允,10,婷允,鄭,1994,-0.717430,-3.676241,5.471447,0.836555,-5.224080,...,1,0,0,0,0,0,0,0,0,0
302428,賴韋芊,10,韋芊,賴,1994,-3.257123,-0.525480,2.698673,5.254115,-1.320439,...,1,0,0,0,0,0,0,0,0,0


In [43]:
sampled_df = add_gender_feature(sampled_df)

In [99]:
sampled_df.describe()

,BirthYear,message,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,...,Zodiac_羊,Zodiac_虎,Zodiac_蛇,Zodiac_豬,Zodiac_雞,Zodiac_馬,Zodiac_鼠,Zodiac_龍,Male_prob,Female_prob
count,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,...,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000,301667.000000
mean,5.493978,1969.555387,-1.351977,-1.096880,0.157860,0.906628,-1.467591,-1.313604,1.034371,1.670677,...,0.070366,0.074098,0.089373,0.082694,0.101509,0.094966,0.085873,0.080466,0.415121,0.023585
std,2.870907,14.377695,3.664740,3.364497,3.591505,3.370239,3.099991,3.187149,3.342380,3.257508,...,0.255763,0.261931,0.285282,0.275419,0.302002,0.293168,0.280177,0.272014,0.427489,0.089865
min,1.000000,1945.000000,-14.552404,-14.552404,-15.155359,-15.155359,-12.120364,-12.120364,-12.768916,-13.059869,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1957.000000,-3.963522,-3.267654,-2.470819,-1.456622,-3.143640,-3.514728,-0.779251,-0.272535,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,1969.000000,-1.203078,-1.118575,0.191441,1.094386,-1.766556,-1.684384,1.192217,1.869913,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.187500,0.000000
75%,8.000000,1982.000000,1.057822,0.880691,2.914812,3.448002,0.364840,0.566860,3.313275,3.917055,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.960938,0.015625
max,10.000000,1994.000000,14.441405,13.809453,13.278703,14.611219,11.268673,12.577226,15.600971,15.600971,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(x)測試一些內容

In [102]:
Counter(list(sampled_df[ sampled_df['Zodiac_羊']==1].BirthYear))

Counter({3: 5979, 5: 4925, 7: 8895, 10: 1428})

In [105]:
Counter(list(sampled_df[ sampled_df['Zodiac_雞']==1].BirthYear))

Counter({1: 4335, 3: 6623, 5: 6976, 8: 5784, 10: 6904})

In [106]:
Counter(list(sampled_df[ sampled_df['Zodiac_龍']==1].BirthYear))

Counter({2: 6021, 4: 5385, 7: 6782, 9: 6086})

In [93]:
y_feature = 'BirthYear'

In [65]:
        
        test_name_dict_list= get_test_name_dict_list(test_x.index,sampled_df,'BirthYear')
        test_name_year_dict_list =  get_test_name_dict_list(test_x.index,sampled_df,'message')
        print('test dataset 有',len(test_x),'個名字,',len(test_name_dict_list),'種名字')    
        test_name_list = sampled_df.loc[test_x.index].FirstName.tolist()

        dist_error=0
        Multi_accuracy=0
        random_dist_error=0
        single_year_error=0
        year_error = [0 for x in range(10) ]
        year_count = [0 for x in range(10) ]
        random_year_error = [0 for x in range(10) ]
        random_score=0
        for test_no,year in enumerate(predictions):
            name = test_name_list[test_no]
            if name in test_name_dict_list:
                year_count[ len(test_name_dict_list[name])-1 ]+=1

                if year in test_name_dict_list[name]:
                    Multi_accuracy+=1
                else:
                    #計算錯誤，把平均絕對值誤差加上去
                    error = get_min_distance(year,test_name_year_dict_list[name]) 
                    dist_error += error
                    
                    year_error[ len(test_name_dict_list[name])-1 ]+=error

                if 1:
                    random_year = random.randint(1,10)
                    if random_year in test_name_dict_list[name]:
                        random_score+=1
                    else:
                        error = get_min_distance(random_year,test_name_year_dict_list[name])
                        random_dist_error += error
                        random_year_error[ len(test_name_dict_list[name])-1 ] += error
        
        Multi_accuracy/=len(predictions)                   
        print ("Train Accuracy :: ", accuracy_score(train_y, trained_model.predict(train_x)))
        print ("Test Accuracy  :: ", Multi_accuracy)
        print('平均年份絕對值誤差::',dist_error/len(predictions) )
                        

test dataset 有 81444 個名字, 41124 種名字
Train Accuracy ::  0.2918725497934591
Test Accuracy  ::  0.429902755267423
平均年份絕對值誤差:: 8.758619419478414


In [51]:
x_feature = get_x_feature(['Zodiac'], sampled_df.columns)

In [110]:
        dev_df = sampled_df.sample( n= len(sampled_df)//10, frac=None , replace=False, weights=None, random_state=None, axis=0)
        train_x, test_x, train_y, test_y = split_dataset( sampled_df.drop(dev_df.index), 0.7, x_feature, y_feature)
       

In [117]:
        trained_model = random_forest_classifier(train_x, train_y.values.reshape(-1, 1).ravel(), estimators_num = 64 , min_samples_leaf_num = 1)


estimators_num =  64 min_samples_leaf_num =  1 Training Data len =  190050


In [118]:
predictions = trained_model.predict(test_x)

In [141]:
Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
    Multi_RFC_metrics(train_x , train_y  , trained_model )

Accuracy ::  0.29271244409365954
macro precision_score ::  0.20176196977914743
micro precision_score ::  0.29271244409365954


C:\Users\n7912\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train recall_score ::  0.29294787760240504
Test recall_score ::  0.29271244409365954
macro F1_score ::  0.23421439780466838
micro F1_score ::  0.29271244409365954


In [123]:
        cf_matrix = confusion_matrix(test_y, trained_model.predict(test_x))
        TN = cf_matrix[0][0]
        FP = cf_matrix[0][1]
        FN = cf_matrix[1][0]
        TP = cf_matrix[1][1]
        precison = TP / (TP + FP)
        recall = TP / (TP + FN)
        accuracy = (TP + TN) / (TN + FP + FN + TP)
        F1 = 2*precison*recall / (precison + recall)
        print("Accuracy :: ", round( accuracy, 4) )
        print("Precision_score :: ", round( precison,4)) 
        print("Recall_score :: ", round(recall,4))
        print("F1_score :: ", round(F1,4) )

Accuracy ::  1.0
Precision_score ::  nan
Recall_score ::  nan
F1_score ::  nan


<ipython-input-123-083ec8b0661e>:6: RuntimeWarning: invalid value encountered in longlong_scalars
  precison = TP / (TP + FP)
<ipython-input-123-083ec8b0661e>:7: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = TP / (TP + FN)


In [ ]:
w_io = open("./Training/result_all.csv", "w")
w_io.write("Type,lens,feature,accuracy,macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1\n")

result_io = open("./Training/result.csv", "w")
result_io.write("feature,lens,accuracy,macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1\n")



for i,feature in enumerate(feature_combinations):
    
    if type(feature)==str:
        x_feature = get_x_feature ([feature] , sampled_df.columns)
    else:
        x_feature = get_x_feature (feature , sampled_df.columns)
    f = ''.join( [ x[0].upper() for x in feature] ).upper()

    print("Combination",i,"Training feature category:",feature)
    print("len of x_feature:",len(x_feature))
    tmp1,tmp2,tmp3,tmp4 = [] , [] , [] , []
    tmp5,tmp6,tmp7 = [] , [] , [] 
    Validation_times = 3
    
    Accuracy_list = []
    macro_precision_list = []
    micro_precision_list = []
    macro_recall_list = []
    micro_recall_list = []
    macro_F1_list = []
    micro_F1_list = []
    avg_dist_error = 0
    avg_random_score=0
    avg_random_dist_error=0
    avg_single_year_errorr=0
    avg_year_error = [0 for x in range(10) ]
    
    for test_time in range(Validation_times):
        random_score=0
        dev_df = sampled_df.sample( n= len(sampled_df)//10, frac=None , replace=False, weights=None, random_state=None, axis=0)
        train_x, test_x, train_y, test_y = split_dataset( sampled_df.drop(dev_df.index), 0.7, x_feature, y_feature)
        
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y.values.reshape(-1, 1).ravel(), estimators_num = 64 , min_samples_leaf_num = 1)
        print('Finished training')
        predictions = trained_model.predict(test_x)
        print('Finished prdeiction')
        
        print("Train Metrics")
        print("train_x len ",len(train_x))
        Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
        Multi_RFC_metrics(train_x , train_y  , trained_model )
        w_io.write("Train,"+f+",{},{},{},{},{},{},{},{}\n".format(len(x_feature), Accuracy, macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1))
        
        print("\nTest Metrics")
        print("Test len ",len(test_x))
        Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
        Multi_RFC_metrics(test_x , test_y  , trained_model )
        w_io.write("Test,"+f+",{},{},{},{},{},{},{},{}\n".format(len(x_feature), Accuracy, macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1))
        
        print("\nDevelopment Metrics")
        print("Development len ",len(dev_df))
        Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
        Multi_RFC_metrics(dev_df[x_feature] , dev_df[y_feature]  , trained_model )
        
        w_io.write("Develop,"+f+",{},{},{},{},{},{},{},{}\n".format(len(x_feature), Accuracy, macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1))
        tmp1+=[Accuracy]
        tmp2+=[macro_precision]
        tmp3+=[micro_precision]
        tmp4+=[macro_recall]
        tmp5+=[micro_recall]
        tmp6+=[macro_F1]
        tmp7+=[micro_F1]
    avg_Accuracy = round( np.array(tmp1).mean() , 4)
    avg_macro_precision = round( np.array(tmp2).mean() ,4)
    avg_micro_precision = round( np.array(tmp3).mean() ,4)

    avg_macro_recall = round( np.array(tmp4).mean(),4)
    avg_micro_recall = round( np.array(tmp5).mean(),4)

    avg_macro_F1 = round( np.array(tmp6).mean(),4)
    avg_micro_F1 = round( np.array(tmp7).mean(),4)

    Accuracy_list+=[avg_Accuracy]
    macro_precision_list = [avg_macro_precision]
    micro_precision_list = [avg_micro_precision]
    macro_recall_list = [avg_macro_recall]
    micro_recall_list = [avg_micro_recall]
    macro_F1_list = [avg_macro_F1]
    micro_F1_list = [avg_micro_F1]
    
    line = f+",{},{},{},{},{},{},{},{}\n"\
        .format(len(x_feature),avg_Accuracy, avg_macro_precision,avg_micro_precision,\
            avg_macro_recall ,avg_micro_recall , avg_macro_F1, avg_micro_F1)
    print("Avg ",line)
    result_io.write(line)
    '''
    FileName = f + "_age_RFC_model.pkl"
    
    with open('./TrainedModel/'+ FileName , 'wb') as handle:
          pickle.dump(trained_model, handle)
          print("\Output model Done.")
    '''
    
w_io.close()
result_io.close()

In [76]:
sampled_df.loc[test_x.index]

,name,BirthYear,FirstName,LastName,message,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,...,Zodiac_羊,Zodiac_虎,Zodiac_蛇,Zodiac_豬,Zodiac_雞,Zodiac_馬,Zodiac_鼠,Zodiac_龍,Male_prob,Female_prob
199583,高藝珊,7,藝珊,高,1979,-2.685985,-2.786899,-4.280315,5.125224,3.911163,...,1,0,0,0,0,0,0,0,0.953125,0.015625
27256,蔡家友,1,家友,蔡,1949,-0.024527,-3.752073,1.429394,-9.503437,-5.721419,...,0,0,0,0,0,0,0,0,0.117188,0.062500
11345,王年陽,1,年陽,王,1949,2.663864,1.868513,-4.532257,4.228958,3.684875,...,0,0,0,0,0,0,0,0,0.242188,0.070312
32126,潘岳濂,2,岳濂,潘,1952,0.238576,-2.645497,1.591030,0.493324,-4.728544,...,0,0,0,0,0,0,0,1,0.140625,0.000000
240195,宋欣怡,8,欣怡,宋,1983,-2.398280,-2.622028,1.651978,3.646321,-1.263631,...,0,0,0,1,0,0,0,0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43908,許群育,2,群育,許,1953,-5.338393,-5.028121,-1.367814,2.607866,-3.066059,...,0,0,1,0,0,0,0,0,0.000000,0.000000
52630,吳思維,2,思維,吳,1950,2.650452,4.150918,0.494862,2.034525,-6.060824,...,0,1,0,0,0,0,0,0,0.000000,0.000000
151452,林明郎,6,明郎,林,1972,-12.395959,-2.260112,-0.514620,-0.405574,-1.830258,...,0,0,0,0,0,0,1,0,0.001379,0.026786
71245,楊建國,3,建國,楊,1959,8.122230,3.000507,-3.091460,-0.890834,-9.115824,...,0,0,0,1,0,0,0,0,0.000000,0.000000


In [77]:
test_y

199583    7
27256     1
11345     1
32126     2
240195    8
         ..
43908     2
52630     2
151452    6
71245     3
59922     2
Name: BirthYear, Length: 81444, dtype: uint8

(5-2) Training for all combination 評量數據

In [19]:
w_io = open("./Training/result_all.csv", "w")
w_io.write("Type,lens,feature,accuracy,macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1\n")

result_io = open("./Training/result.csv", "w")
result_io.write("feature,lens,accuracy,multi_ans_accuracy,avg_dist,macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1\n")


Validation_times = 3
for i,feature in enumerate(feature_combinations):
    
    if 5(feature)==str:
        x_feature = get_x_feature ([feature] , sampled_df.columns)
    else:
        x_feature = get_x_feature (feature , sampled_df.columns)
    f = ''.join( [ x[0].upper() for x in feature] ).upper()

    print("Combination",i,"Training feature category:",feature)
    print("len of x_feature:",len(x_feature))
    tmp1,tmp2,tmp3,tmp4 = [] , [] , [] , []
    tmp5,tmp6,tmp7 = [] , [] , [] 
    
    Accuracy_list = []
    macro_precision_list = []
    micro_precision_list = []
    macro_recall_list = []
    micro_recall_list = []
    macro_F1_list = []
    micro_F1_list = []
    
    Train_accuracy_score=0
    Test_accuracy_score=0
    avg_dist_error = 0
    avg_single_year_errorr=0
    avg_year_error = [0 for x in range(10) ]
    
    for test_time in range(Validation_times):
        random_score=0
        dev_df = sampled_df.sample( n= len(sampled_df)//10, frac=None , replace=False, weights=None, random_state=None, axis=0)
        train_x, test_x, train_y, test_y = split_dataset( sampled_df.drop(dev_df.index), 0.7, x_feature, y_feature)
        
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y.values.reshape(-1, 1).ravel(), estimators_num = 64 , min_samples_leaf_num = 1)
        print('Finished training')
        predictions = trained_model.predict(test_x)
        print('Finished prdeiction')
        
        test_name_dict_list= get_test_name_dict_list(test_x.index,sampled_df,'BirthYear')
        test_name_year_dict_list =  get_test_name_dict_list(test_x.index,sampled_df,'message')
        print('test dataset 有',len(test_x),'個名字,',len(test_name_dict_list),'種名字')    

        dist_error=0
        Multi_accuracy=0
        random_dist_error=0
        single_year_error=0
        year_error = [0 for x in range(10) ]
        year_count = [0 for x in range(10) ]
        random_year_error = [0 for x in range(10) ]    
        test_name_list = sampled_df.loc[test_x.index].FirstName.tolist()
        for test_no,year in enumerate(predictions):
            name = test_name_list[test_no]
            if name in test_name_dict_list:
                if year in test_name_dict_list[name]:
                    Multi_accuracy+=1
                else:
                    #計算錯誤，把平均絕對值誤差加上去
                    error = get_min_distance(year,test_name_year_dict_list[name]) 
                    dist_error += error
                    
            else:
                print(name)
        Multi_accuracy/=len(predictions)   
        print ("Multi_Test Accuracy  :: ", Multi_accuracy)
        print('平均年份絕對值誤差::',dist_error/len(predictions) )
           
        avg_dist_error+=dist_error/len(predictions)

                   
        print("Train Metrics")
        print("train_x len ",len(train_x))
        Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
        Multi_RFC_metrics(train_x , train_y  , trained_model )
        w_io.write("Train,"+f+",{},{},{},{},{},{},{},{}\n".format(len(x_feature), Accuracy, macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1))
        
        Train_accuracy_score+=Accuracy
        Test_accuracy_score+= Multi_accuracy
                
        print("\nTest Metrics")
        print("Test len ",len(test_x))
        Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
        Multi_RFC_metrics(test_x , test_y  , trained_model )
        w_io.write("Test,"+f+",{},{},{},{},{},{},{},{}\n".format(len(x_feature), Accuracy, macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1))
        
        print("\nDevelopment Metrics")
        print("Development len ",len(dev_df))
        Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
        Multi_RFC_metrics(dev_df[x_feature] , dev_df[y_feature]  , trained_model )
        
        w_io.write("Develop,"+f+",{},{},{},{},{},{},{},{}\n".format(len(x_feature), Accuracy, macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1))
        tmp1+=[Accuracy]
        tmp2+=[macro_precision]
        tmp3+=[micro_precision]
        tmp4+=[macro_recall]
        tmp5+=[micro_recall]
        tmp6+=[macro_F1]
        tmp7+=[micro_F1]
        
        
    avg_Accuracy = round( np.array(tmp1).mean() , 4)
    avg_macro_precision = round( np.array(tmp2).mean() ,4)
    avg_micro_precision = round( np.array(tmp3).mean() ,4)

    avg_macro_recall = round( np.array(tmp4).mean(),4)
    avg_micro_recall = round( np.array(tmp5).mean(),4)

    avg_macro_F1 = round( np.array(tmp6).mean(),4)
    avg_micro_F1 = round( np.array(tmp7).mean(),4)

    Accuracy_list+=[avg_Accuracy]
    macro_precision_list = [avg_macro_precision]
    micro_precision_list = [avg_micro_precision]
    macro_recall_list = [avg_macro_recall]
    micro_recall_list = [avg_micro_recall]
    macro_F1_list = [avg_macro_F1]
    micro_F1_list = [avg_micro_F1]
    
    if(Validation_times!=1):
        Train_accuracy_score = round(Train_accuracy_score / Validation_times,4 )
        Test_accuracy_score = round(Test_accuracy_score / Validation_times,4 )
        avg_dist_error = round(avg_dist_error/Validation_times , 4)
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ",Test_accuracy_score)
        print('Average 平均年份絕對值誤差::', avg_dist_error)
    
    line = f+",{},{},{},{},{},{},{},{},{},{}\n"\
        .format(len(x_feature),avg_Accuracy,Test_accuracy_score,avg_dist_error, avg_macro_precision,avg_micro_precision,\
            avg_macro_recall ,avg_micro_recall , avg_macro_F1, avg_micro_F1)
    print("Avg ",line)
    result_io.write(line)
    '''
    FileName = f + "_age_RFC_model.pkl"
    
    with open('./TrainedModel/'+ FileName , 'wb') as handle:
          pickle.dump(trained_model, handle)
          print("\Output model Done.")
    '''
    
w_io.close()
result_io.close()

<>:11: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:11: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<ipython-input-19-e43f355fdb62>:11: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  if 5(feature)==str:


TypeError: 'int' object is not callable

(5-1B) baseline dataset preprocess, train之前的baseline dataset前先處理一下

In [21]:
small_df.head(5)

,FirstName,message,name,FN1,FN2,FN1_muin,FN2_muin,BirthYear,GuessedGender,BFN12P,...,FirstName1_ratio_6,FirstName2_ratio_6,FirstName1_ratio_7,FirstName2_ratio_7,FirstName1_ratio_8,FirstName2_ratio_8,FirstName1_ratio_9,FirstName2_ratio_9,FirstName1_ratio_10,FirstName2_ratio_10
0,秀珠,1947,吳秀珠,70,596,iù,ū,1,0,1,...,0.014783,0.004958,0.005215,0.001011,0.004419,0.000620,0.002103,0.000166,0.001614,0.000071
1,豐博,1945,林豐博,277,72,ēng,ó,1,1,6,...,0.002408,0.001052,0.000821,0.000999,0.000986,0.002212,0.000701,0.002531,0.000680,0.003256
2,義勝,1945,康義勝,650,156,ì,èng,1,1,1,...,0.003228,0.003177,0.001718,0.001045,0.001546,0.002305,0.000899,0.001726,0.000665,0.001187
3,麗,1946,吳麗,-1,203,-1,ì,1,0,6,...,0.002688,0.015062,0.001453,0.002185,0.004345,0.002212,0.007338,0.001254,0.012524,0.000856
5,世慶,1947,林世慶,537,269,ì,ìng,1,1,6,...,0.004557,0.004066,0.004241,0.001383,0.003323,0.002356,0.002243,0.001752,0.001613,0.001902


In [22]:
x_train = small_df.loc[train_index]

In [23]:
x_train

,FirstName,message,name,FN1,FN2,FN1_muin,FN2_muin,BirthYear,GuessedGender,BFN12P,...,FirstName1_ratio_6,FirstName2_ratio_6,FirstName1_ratio_7,FirstName2_ratio_7,FirstName1_ratio_8,FirstName2_ratio_8,FirstName1_ratio_9,FirstName2_ratio_9,FirstName1_ratio_10,FirstName2_ratio_10
190169,馨尹,1976,張馨尹,528,224,īn,ǐn,7,0,9,...,0.000888,0.000089,0.001200,0.000487,0.002626,0.000620,0.003040,0.000828,0.003053,0.000584
55538,康宜,1951,李康宜,37,16,āng,í,2,1,8,...,0.000273,0.003748,0.000195,0.005055,0.000122,0.007838,0.000177,0.007820,0.000178,0.008235
135875,傳宗,1965,詹傳宗,720,161,uán,ōng,5,1,3,...,0.000590,0.005546,0.000941,0.004093,0.000775,0.005822,0.000350,0.006235,0.000305,0.004762
211797,俊廷,1984,劉俊廷,18,22,ùn,íng,8,1,10,...,0.009312,0.000936,0.007463,0.001556,0.009837,0.004178,0.006459,0.006032,0.006775,0.007347
170348,超群,1979,張超群,884,607,āo,ún,7,1,8,...,0.000671,0.000863,0.000560,0.001124,0.000829,0.001574,0.000641,0.001732,0.000368,0.001192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276658,欣儀,1994,林欣儀,78,7,īn,í,10,0,8,...,0.003144,0.002155,0.004491,0.002114,0.007807,0.004312,0.007694,0.004147,0.007228,0.005356
101464,文俞,1964,林文俞,28,726,én,ú,4,0,9,...,0.021664,0.000201,0.017651,0.000306,0.015007,0.000508,0.012276,0.001028,0.011234,0.000997
287707,映均,1993,陳映均,445,595,ìng,ūn,10,0,10,...,0.000144,0.000120,0.000441,0.000609,0.000764,0.001815,0.000997,0.002280,0.001622,0.003698
293817,靜慧,1971,蘇靜慧,217,124,ìng,ì,6,0,7,...,0.004703,0.011040,0.004020,0.006169,0.004940,0.009741,0.003851,0.006792,0.003109,0.004556


In [24]:
for f in ['w2v', 'phonetic', 'fortune_map', 'Zodiac', 'radical' ,'gender']:
    x_feature = get_x_feature ([f] , small_df.columns)

    print(len(x_feature))

200
63
31
0
315
2


In [25]:
feature = get_x_feature (['w2v', 'phonetic', 'fortune_map', 'Zodiac', 'radical' ,'gender'] , small_df.columns)
feature+=['BirthYear','FirstName','LastName','name','message','FN1','FN2']
small_df = small_df[feature]

In [26]:
def Get_Light_NameDF(df):
    type_dic = {}
    for w2v_col in get_x_feature(['w2v'],df):
        type_dic[ w2v_col ] = 'float32'

    for col in get_x_feature(['phonetic','fortune_map','Zodiac','radical'],df):
        type_dic[ col ] = 'int8'
    df = df.astype( type_dic )
    return df

In [30]:
small_df.dtypes

Male_prob      float64
Female_prob    float64
FN1_wv_0       float32
FN2_wv_0       float32
FN1_wv_1       float32
                ...   
LastName        object
name            object
message          int64
FN1              int64
FN2              int64
Length: 618, dtype: object

In [28]:
small_df = Get_Light_NameDF(small_df)

In [31]:
train_y = small_df.loc[train_index]
y_feature = 'BirthYear'
train_y = train_y[y_feature]

test_y = small_df.loc[test_index]
test_y = test_y[y_feature]

test_name_dict_list= get_test_name_dict_list(test_index,small_df,'BirthYear')
test_name_year_dict_list =  get_test_name_dict_list(test_index,small_df,'message')
test_name_list = small_df.loc[test_index].FirstName.tolist()

print('test dataset 有',len(test_index),'個名字,',len(test_name_dict_list),'種名字')  

test dataset 有 90730 個名字, 43608 種名字


In [32]:
def get_multi_Answer_Result(predictions , test_name_dict_list , test_name_list ):
    TP=0
    FP=0
    TN=0
    FN=0        
    dist_error=0
    Multi_accuracy=0
    
    for test_no,year in enumerate(predictions):
            name = test_name_list[test_no]
            if name in test_name_dict_list:
                #若test name 所有叫宜欣的年齡分布是   1=3個  2=6個  7=8個  9=1個人
                #test name 所有叫宜欣的年齡分布是   1=3個  2=6個  7=8個  9=1個人
                if year in test_name_dict_list[name]:
                    Multi_accuracy+=1
                    
                #猜其他的就錯，算年齡誤差
                else:
                    #計算誤差，把平均絕對值誤差加上去
                    dist_error +=  get_min_distance(year,test_name_year_dict_list[name]) 
            else:
                print('It must be something wrong,',name)
    Multi_accuracy = round( Multi_accuracy / len(predictions) ,4 )
    dist_error = round( dist_error / len(predictions) ,4 )
    return Multi_accuracy, dist_error

In [132]:
predictions

array([4, 8, 9, ..., 2, 8, 9], dtype=int64)

try data augmentation  把前後字名字對掉增加資料量

In [27]:
def FN_augmentation( df ):
    rename_dic = {}
    for col in (df.columns):
        if "FN1" in col:
            rename_dic[col] = col.replace("FN1","FN2")
        elif  "FN2" in col:
            rename_dic[col] = col.replace("FN2","FN1")
    df = pd.concat([df, df.rename(columns = rename_dic)], axis=0)
    
    return df.fillna(0)

(5-2B)Training for all combination - baseline dataset

In [146]:
small_df.loc[train_index]

,Male_prob,Female_prob,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,...,Zodiac_羊,Zodiac_虎,Zodiac_蛇,Zodiac_豬,Zodiac_雞,Zodiac_馬,Zodiac_鼠,Zodiac_龍,FN2_radical_-1,FN2_sonin_-1
190169,0.011719,0.988281,-0.805664,-5.078125,-0.117737,1.883789,-3.267578,-0.840820,2.738281,0.907715,...,0,0,0,0,0,0,0,1,0.0,0.0
55538,0.722656,0.277344,-1.362305,-2.677734,4.132812,-0.251709,3.513672,-3.068359,-0.755859,1.109375,...,0,0,0,0,0,0,0,0,0.0,0.0
135875,0.988281,0.011719,0.838379,-0.566406,-8.953125,1.168945,-4.812500,1.902344,-1.249023,-3.203125,...,0,0,1,0,0,0,0,0,0.0,0.0
211797,1.000000,0.000000,-6.195312,-0.523926,-0.321045,3.423828,-2.705078,-3.394531,3.626953,-2.505859,...,0,0,0,0,0,0,1,0,0.0,0.0
170348,1.000000,0.000000,-3.597656,-5.339844,-5.960938,-1.368164,-0.759277,-3.066406,-0.893066,3.638672,...,1,0,0,0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276658,0.000000,0.996094,0.737305,-2.398438,4.535156,1.652344,-0.042694,-1.263672,2.083984,3.888672,...,0,0,0,0,0,0,0,0,0.0,0.0
101464,0.218750,0.781250,-1.850586,2.777344,2.814453,0.408203,-2.775391,-1.684570,0.934082,-0.272461,...,0,0,0,0,0,0,0,1,0.0,0.0
287707,0.062500,0.933594,-5.968750,-3.263672,-5.535156,-0.406250,2.042969,-4.085938,-1.169922,7.835938,...,0,0,0,0,1,0,0,0,0.0,0.0
293817,0.003906,0.996094,-5.238281,-1.043945,3.181641,-0.432861,-2.939453,-0.059998,3.542969,2.191406,...,0,0,0,1,0,0,0,0,0.0,0.0


In [29]:
train = small_df.loc[train_index]

In [30]:
train = FN_augmentation(train)

In [35]:
w_io = open("./Training/result_all.csv", "w")
w_io.write("Type,lens,feature,accuracy,multi_ans_accuracy,avg_dist,macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1\n")

result_io = open("./Training/result.csv", "w")
result_io.write("feature,lens,accuracy,multi_ans_accuracy,avg_dist,macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1\n")

y_feature = 'BirthYear'
    
Validation_times = 5
for i,feature in enumerate(feature_combinations):
    feature = 'uni-gram'
    if type(feature)==str:
        x_feature = get_x_feature ([feature] , small_df.columns)
    else:
        x_feature = get_x_feature (feature , small_df.columns)
    f = ''.join( [ x[0].upper() for x in feature] ).upper()

    #train_x= train[x_feature]
    #train_y= train[y_feature]

    test_x = small_df.loc[test_index]
    test_x= test_x[x_feature]
    
    train_x = small_df.loc[train_index]
    train_x= train_x[x_feature]
    #train_x = FN_augmentation(train_x)
    
    
    print("Combination",i,"Training feature category:",feature)
    print("len of x_feature:",len(x_feature))
    tmp1,tmp2,tmp3,tmp4 = [] , [] , [] , []
    tmp5,tmp6,tmp7 = [] , [] , [] 
    
    Accuracy_list = []
    macro_precision_list = []
    micro_precision_list = []
    macro_recall_list = []
    micro_recall_list = []
    macro_F1_list = []
    micro_F1_list = []
    
    Train_accuracy_score=0
    Test_accuracy_score=0
    avg_dist_error = 0
    avg_single_year_errorr=0
    avg_year_error = [0 for x in range(10) ]
    
    for test_time in range(Validation_times):
        #dev_df = sampled_df.sample( n= len(sampled_df)//10, frac=None , replace=False, weights=None, random_state=None, axis=0)
        #train_x, test_x, train_y, test_y = split_dataset( sampled_df.drop(dev_df.index), 0.7, x_feature, y_feature)
        
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y.values.reshape(-1, 1).ravel(), estimators_num = 64 , min_samples_leaf_num = 1)
        print('Finished training')
        predictions = trained_model.predict(test_x)
        print('Finished prdeiction')
        
        #test_name_dict_list= get_test_name_dict_list(test_x.index,sampled_df,'BirthYear')
        #test_name_year_dict_list =  get_test_name_dict_list(test_x.index,sampled_df,'message')
        #print('test dataset 有',len(test_x),'個名字,',len(test_name_dict_list),'種名字')    

        Multi_accuracy, dist_error = get_multi_Answer_Result(predictions , test_name_dict_list , test_name_list)
        print ("Multi_Test Accuracy  :: ", Multi_accuracy)
        print('平均年份絕對值誤差::',dist_error)
        avg_dist_error+=dist_error
                   
        print("Train Metrics")
        print("train_x len ",len(train_x))
        Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
        Multi_RFC_metrics(train_x , train_y  , trained_model )
        w_io.write("Train,"+f+",{},{},{},{},{},{},{},{},{},{}\n".format(\
            len(x_feature), Accuracy, 'NA','NA',\
                macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1))
        
        Train_accuracy_score+=Accuracy
        Test_accuracy_score+= Multi_accuracy
                
        print("\nTest Metrics")
        print("Test len ",len(test_x))
        Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
        Multi_RFC_metrics(test_x , test_y  , trained_model )
        w_io.write("Test,"+f+",{},{},{},{},{},{},{},{},{},{}\n".format(\
            len(x_feature), Accuracy,Multi_accuracy,dist_error,\
                macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1))
        
        '''
        print("\nDevelopment Metrics")
        print("Development len ",len(dev_df))
        Accuracy, macro_precision , micro_precision , macro_recall , micro_recall , macro_F1,micro_F1 = \
        Multi_RFC_metrics(dev_df[x_feature] , dev_df[y_feature]  , trained_model )
        
        w_io.write("nDevelopment,"+f+",{},{},{},{},{},{},{},{}\n".format(len(x_feature), Accuracy, macro_precision,micro_precision,macro_recall,micro_recall,macro_F1,micro_F1))
        '''
        tmp1+=[Accuracy]
        tmp2+=[macro_precision]
        tmp3+=[micro_precision]
        tmp4+=[macro_recall]
        tmp5+=[micro_recall]
        tmp6+=[macro_F1]
        tmp7+=[micro_F1]
        
        
    avg_Accuracy = round( np.array(tmp1).mean() , 4)
    avg_macro_precision = round( np.array(tmp2).mean() ,4)
    avg_micro_precision = round( np.array(tmp3).mean() ,4)

    avg_macro_recall = round( np.array(tmp4).mean(),4)
    avg_micro_recall = round( np.array(tmp5).mean(),4)

    avg_macro_F1 = round( np.array(tmp6).mean(),4)
    avg_micro_F1 = round( np.array(tmp7).mean(),4)

    Accuracy_list+=[avg_Accuracy]
    macro_precision_list = [avg_macro_precision]
    micro_precision_list = [avg_micro_precision]
    macro_recall_list = [avg_macro_recall]
    micro_recall_list = [avg_micro_recall]
    macro_F1_list = [avg_macro_F1]
    micro_F1_list = [avg_micro_F1]
    
    if(Validation_times!=1):
        Train_accuracy_score = round(Train_accuracy_score / Validation_times,4 )
        Test_accuracy_score = round(Test_accuracy_score / Validation_times,4 )
        avg_dist_error = round(avg_dist_error/Validation_times , 4)
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ",Test_accuracy_score)
        print('Average 平均年份絕對值誤差::', avg_dist_error)
    
    line = f+",{},{},{},{},{},{},{},{},{},{}\n"\
        .format(len(x_feature),avg_Accuracy,Test_accuracy_score,avg_dist_error, avg_macro_precision,avg_micro_precision,\
            avg_macro_recall ,avg_micro_recall , avg_macro_F1, avg_micro_F1)
    print("Avg ",line)
    result_io.write(line)
    break
    '''
    FileName = f + "_age_RFC_model.pkl"
    
    with open('./TrainedModel/'+ FileName , 'wb') as handle:
          pickle.dump(trained_model, handle)
          print("\Output model Done.")
    '''
    
w_io.close()
result_io.close()

Combination 0 Training feature category: uni-gram
len of x_feature: 2
estimators_num =  64 min_samples_leaf_num =  1 Training Data len =  211702
Finished training
Finished prdeiction
Multi_Test Accuracy  ::  0.5154
平均年份絕對值誤差:: 4.9711
Train Metrics
train_x len  211702
Accuracy ::  0.5314451445900369
macro precision_score ::  0.5319600141909395
micro precision_score ::  0.5314451445900369
Train recall_score ::  0.5316295357145824
Test recall_score ::  0.5314451445900369
macro F1_score ::  0.5314928736280679
micro F1_score ::  0.5314451445900369

Test Metrics
Test len  90730
Accuracy ::  0.19863330761600353
macro precision_score ::  0.19801242908531985
micro precision_score ::  0.19863330761600353
Train recall_score ::  0.1990004419450266
Test recall_score ::  0.19863330761600353
macro F1_score ::  0.19837359992896206
micro F1_score ::  0.1986333076160035
estimators_num =  64 min_samples_leaf_num =  1 Training Data len =  211702
Finished training
Finished prdeiction
Multi_Test Accuracy  :

In [80]:
predictions = trained_model.predict(test_x)
predictions

array([8, 4, 5, ..., 7, 6, 3], dtype=uint8)

In [138]:
w_io.close()
result_io.close()

(5-3) metric 評量測試

In [37]:
def Multi_RFC_metrics(  data_x , data_y , trained_model ):
        predicted = trained_model.predict(data_x)
        accuracy = accuracy_score(data_y,predicted)
        print("Accuracy :: ", accuracy)
        
        macro_precision = precision_score(data_y, predicted ,average='macro')
        micro_precision = precision_score(data_y, predicted,average='micro')
        print("macro precision_score :: ", macro_precision)
        print("micro precision_score :: ", micro_precision)
        
        macro_recall = recall_score(data_y, predicted ,average='macro')
        micro_recall = recall_score(data_y, predicted,average='micro')        
        print("Train recall_score :: ", macro_recall)
        print("Test recall_score :: ", micro_recall)
        
        macro_f1 = f1_score(data_y, predicted,average='macro')
        micro_f1 = f1_score(data_y, predicted,average='micro')
        print("macro F1_score :: ", macro_f1)
        print("micro F1_score :: ", micro_f1)


        return round( accuracy, 4) , round( macro_precision,4) , round(micro_precision ,4) ,\
                round(macro_recall,4) , round(micro_precision,4) \
                ,  round(macro_f1,4) ,round(micro_f1,4)



In [97]:
target_names = [ '女性' ,'男性']

print("report:\n",classification_report(test_y, predictions ,target_names= target_names))

report:
               precision    recall  f1-score   support

          女性       0.88      0.84      0.86     15534
          男性       0.89      0.91      0.90     21862

    accuracy                           0.88     37396
   macro avg       0.88      0.88      0.88     37396
weighted avg       0.88      0.88      0.88     37396



In [ ]:
import datetime
# multi-answer RFC
def basic_RFClassifier_ts(data_df,x_feature,y_feature,Validation_times,mode,train_index,test_index,random_avg_year_error):
    single_year_number=0
    print(datetime.datetime.now())
    #N fold cross-validation
    Train_accuracy_score=0
    Test_accuracy_score=0
    
    avg_dist_error = 0
    avg_random_score=0
    avg_random_dist_error=0
    avg_single_year_errorr=0
    avg_year_error = [0 for x in range(10) ]
    if mode!=2:
        random_avg_year_error = [0 for x in range(10) ]
    for i in range(Validation_times):
        random_score=0
        if mode==0:
            train_x, test_x, train_y, test_y = split_dataset(data_df, 0.7, x_feature, y_feature)
            train_index = train_x.index
            test_index = test_x.index
        else:
            train_x = data_df.loc[train_index][x_feature]
            test_x =data_df.loc[test_index][x_feature]
            train_y = data_df.loc[train_index][y_feature]
            train_y = train_y.values.reshape(-1, 1)
            test_y = data_df.loc[test_index][y_feature]
            test_y = test_y.values.reshape(-1, 1)
        
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y.ravel())
        print('Finished training')
        
        predictions = trained_model.predict(test_x)
        print('Finished prdeiction')
        # Train and Test Accuracy
        if(Validation_times!=1):
            print('Run times:',i+1)

        test_name_dict_list= get_test_name_dict_list(test_index,data_df,'BirthYear')
        test_name_year_dict_list =  get_test_name_dict_list(test_index,data_df,'message')
        print('test dataset 有',len(test_x),'個名字,',len(test_name_dict_list),'種名字')    
        
        test_name_list = data_df.loc[test_index].FirstName.tolist()
        
        dist_error=0
        accuracy=0
        random_dist_error=0
        single_year_error=0
        year_error = [0 for x in range(10) ]
        year_count = [0 for x in range(10) ]
        
        if mode!=2:
            random_year_error = [0 for x in range(10) ]
            
        for test_no,year in enumerate(predictions):
            

            #print(name)
            name = test_name_list[test_no]
            if name in test_name_dict_list:
                year_count[ len(test_name_dict_list[name])-1 ]+=1

                if year in test_name_dict_list[name]:
                    accuracy+=1
                else:
                    #計算錯誤，把平均絕對值誤差加上去
                    error = get_min_distance(year,test_name_year_dict_list[name]) 
                    dist_error += error
                    
                    year_error[ len(test_name_dict_list[name])-1 ]+=error
      
                if mode!=2:
                    random_year = random.randint(1,10)
                    if random_year in test_name_dict_list[name]:
                        random_score+=1
                    else:
                        error = get_min_distance(random_year,test_name_year_dict_list[name])
                        random_dist_error += error
                        random_year_error[ len(test_name_dict_list[name])-1 ] += error
            else:
                print('wtf test name not in test_name_dict_list')
        
        accuracy/=len(predictions)      

        print ("Train Accuracy :: ", accuracy_score(train_y, trained_model.predict(train_x)))
        print ("Test Accuracy  :: ", accuracy)
        print('平均年份絕對值誤差::',dist_error/len(predictions) )
        #print('只在某一年份出現的平均年份絕對值誤差::',single_year_error/single_year_number)
            
        for i in range(len(year_error)):
            avg_year_error[i]+=year_error[i]/year_count[i]
            if mode!=2:
                random_avg_year_error[i] = random_year_error[i]/year_count[i]
                random_avg_year_error[i]+=random_year_error[i]/year_count[i]
            print('名字只出現在共',i+1,'個年份區間的名字比例::',round(year_count[i]/len(predictions),4)*100,'%')
            print('名字只出現在共',i+1,'個年份區間的平均誤差::',round(year_error[i]/year_count[i],4))
            if mode!=2:
                print('random 名字只出現在共',i+1,'個年份區間的平均誤差::',round(random_year_error[i]/year_count[i],4))
        if mode==1:    
            print('Random guess accuracy::',random_score/len(predictions))
            print('Random guess error::',random_dist_error/len(predictions))

        #print (" Confusion matrix\n ", confusion_matrix(test_y, predictions))
        
        Train_accuracy_score+=accuracy_score(train_y, trained_model.predict(train_x))
        Test_accuracy_score+= accuracy
        avg_dist_error+=dist_error/len(predictions)
        avg_random_score+=random_score/len(predictions)
        avg_random_dist_error=random_dist_error/len(predictions)
        
#         calculate_errorr_multi_answer( predictions, test_x ,test_name_dict_list,data_df)
#         validate_error_multi_answer( predictions, data_df,test_y,test_x ,test_name_dict_list)
        
    if(Validation_times!=1):
        Train_accuracy_score/=Validation_times
        Test_accuracy_score/=Validation_times
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ",Test_accuracy_score)
        print('Average 平均年份絕對值誤差::',avg_dist_error/Validation_times )
        for i in range(len(year_error)):
            print('Average 出現在共',i+1,'年份區間的平均誤差::',round(avg_year_error[i]/Validation_times,4))
            print('random 名字只出現在共',i+1,'個年份區間的平均誤差::',round(random_avg_year_error[i]/Validation_times,4))
        if mode!=2:
            print('Average Random guess accuracy::',avg_random_score/Validation_times)
            print('Average Random guess error::',avg_random_dist_error/Validation_times)  
        
        for i in range(9):
            improve_rate = abs(avg_year_error[i]-random_avg_year_error[i])/random_avg_year_error[i]
            print('名字只出現在共',i+1,'個年份區間的的improve rate:',round(improve_rate*100,2),'%')
    return random_avg_year_error

In [ ]:
basic_RFClassifier_ts(small_df,x_feature,y_feature,2,2,train_index,test_index,random_avg_year_error)